In [8]:
import pandas as pd
import time
import numpy as np
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import InvalidSelectorException

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.sentiment import SentimentIntensityAnalyzer

In [9]:
phone_data = pd.read_csv("Final_Phone_Data.csv",index_col='Unnamed: 0')
review_data = pd.read_csv("Final_Review_Data.csv",index_col='Unnamed: 0')
phone_data['Price'] = pd.to_numeric(phone_data['Price'].str.replace("Rs","").str.replace(".","").str.replace(",",""))
phone_data['Company'] = phone_data['Company'].str.lower() 
phone_data['Product Name'] = phone_data['Product Name'].str.lower() 
phone_data['Specifications'] = phone_data['Specifications'].str.lower() 
phone_data['Rating'] = pd.to_numeric(phone_data['Rating'].str.replace('-','0'))
phone_data['Total Ratings'] = pd.to_numeric(phone_data['Total Ratings'].str.replace('No','0'))
phone_data['Total Questions Answered'] = pd.to_numeric(phone_data['Total Questions Answered'].str.replace('-','0'))
phone_data

,Product ID,Company,Product Name,Price,Rating,Total Ratings,Total Questions Answered,Specifications,Link
0,1,samsung,samsung a14 4gb/64gb - 5000 mah battery - 6.6'...,34999,4.4,188,336,"[""• samsung a14 with 4gb/64gb memory.• 5000 ma...",https://www.daraz.pk/products/samsung-a14-4gb6...
1,2,redmi,redmi note 12 8 gb ram + 128gb rom pta approve...,49999,4.7,401,0,['redmi note 12 8gb ram + 128gb rom pta approv...,https://www.daraz.pk/products/redmi-note-12-8-...
2,3,infinix,infinix note 30 pro 8-256 gb pta approved with...,64499,4.7,152,217,"['release date 2023-05-22', 'sim support dual ...",https://www.daraz.pk/products/infinix-note-30-...
3,4,redmi,redmi a2+ - 3gb/64gb - mediatek helio g36 - an...,19399,4.7,367,0,['display: 6.52” hd+ dot drop displayresolutio...,https://www.daraz.pk/products/redmi-a2-3gb64gb...
4,5,realme,realme narzo 50a prime (4gb-128gb) pta approve...,32999,4.2,93,141,"[""'colour"", 'xanh tia chop = lightning blue', ...",https://www.daraz.pk/products/realme-narzo-50a...
...,...,...,...,...,...,...,...,...,...
195,196,infinix,"infinix smart 7 hd , 2gb ram 64gb rom , 8mp ma...",21499,5.0,1,20,['general featuresrelease date2023-04-14sim su...,https://www.daraz.pk/products/7-hd-2gb-64gb-8m...
196,197,tecno,tecno spark 10 ram 4 gb rom 128 gb front camer...,34999,5.0,3,0,['buildosandroid 13 os uihios 12.0 dimensionsn...,https://www.daraz.pk/products/10-4-gb-128-gb-8...
197,198,infinix,infinix zero 30 4g ram 8 gb rom 256 gb front c...,69999,5.0,1,0,['buildosandroid 13 os uixos 13 dimensions164....,https://www.daraz.pk/products/30-4g-8-gb-256-g...
198,199,tecno,"tecno camon 20 pro - 6.67"" inch display - 8gb ...",61899,0.0,0,0,['1 year brand warranty'],https://www.daraz.pk/products/tecno-camon-20-p...


In [10]:
def clean_string(input_string):
    clean_string = re.sub(r'(?<!\d)[^\w\s.](?!\d)', '', input_string)
    clean_string = clean_string.lower()
    # print(clean_string)
    words = word_tokenize(clean_string)
    return words

def clean_string2(input_string):
    clean_string = re.sub(r'(?<!\d)[^\w\s.](?!\d)', '', input_string)
    clean_string = clean_string.lower()
    # print(clean_string)
    # words = word_tokenize(clean_string)
    return clean_string

def contains_Brand(text):
    Brands = ['redmi','apple','infinix','samsung','tecno','xiaomi','vivo','realme','itel','sparx','vgotel','nokia','oppo','combo','zte','imported','y81','vgo','iphone','lg','honor']
    brands = []
    cond = False
    for word in text:
        if word in Brands:
            brands.append(word)
            cond = True
    return cond,brands

def contains_Rating(text):
    words = ['score','ranking','rating','rated']
    for word in text:
        if word in words:
            return True

def contains_digit(text):
    num = []
    num2 = []
    for word in text:
        try:
            if (word.isdigit()):
                num.append(int(word))
            else:
                num2.append(float(word))
        except ValueError:
            pass
    return num,num2

def contains_highest(text):
    dict = ['highest','greater','more','higher','high','top','good','above']
    for word in text:
        if word in dict:
            return True
            
def contains_lowest(text):
    dict = ['lower','lowest','less','lesser','under','below','worse','low']
    for word in text:
        if word in dict:
            return True

def contains_price_low(text):
    dict = ['under','below','lower','lesser']
    for word in text:
        if word in dict:
            return True


def contains_price_high(text):
    dict = ['greater','greater than','above']
    for word in text:
        if word in dict:
            return True
            

In [11]:
def Display_Brands(query):
    bool,brand = contains_Brand(query) # Brand Given or Not
    num6,num7 = contains_digit(query)
    if (bool==True):
        if (len(brand)==1):
            brand = brand[0]
            if (contains_Rating(query)):          # Contains Rating?
                num,num2 = contains_digit(query)       # Checking if Rating specified or not
                if(len(num2)==1 and len(num)==0):                  # only rating given
                    rating = num2[0]
                    if (contains_highest(query)):    # Rating Higher?
                        query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']>rating)]
                        query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Rating Lower?
                        print("Working")
                        query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']<rating)]
                        query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                        return query
                    
                elif(len(num2)==1 and len(num)==1):     # Rating and Price Given
                    price = num[0]
                    rating = num2[0]
                    if (contains_highest(query)):   # Rating Higher?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']>rating) &  (phone_data['Price']<price)]
                            query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']>rating) &  (phone_data['Price']>price)]
                            query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                            return query
                    elif (contains_lowest(query)):  # Rating Lower?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']<rating) &  (phone_data['Price']<price)]
                            query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']<rating) &  (phone_data['Price']>price)]
                            query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                            return query
                    
                
                elif(len(num)==2 and len(num2)==1):   # Rating + Price in between
                    
                    price1 = num[0]
                    price2 = num[1]
                    rating = num2[0]

                    if (contains_highest(query)):   # Rating Higher?
                        query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']>rating) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Rating Lower?
                        query = phone_data[(phone_data['Company'].str.contains(brand)) & (phone_data['Rating']<rating) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                        return query
                        
                elif (len(num)==1 and len(num2)==0):
                    price = num[0]
                    if (contains_highest(query)):   # Rating Higher?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) &  (phone_data['Price']<price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) &  (phone_data['Price']>price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                    elif (contains_lowest(query)):  # Rating Lower?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) &  (phone_data['Price']<price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[(phone_data['Company'].str.contains(brand)) &  (phone_data['Price']>price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                            
                elif (len(num)==2 and len(num2)==0):
                    price1 = num[0]
                    price2 = num[1]

                    if (contains_highest(query)):   # Rating Higher?
                        query = phone_data[(phone_data['Company'].str.contains(brand)) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Rating Lower?
                        query = phone_data[(phone_data['Company'].str.contains(brand)) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    
                    
                else:    # Rating is not specified
                    
                    if (contains_highest(query)):  # High Rated Product?
                        query = phone_data[phone_data['Company'].str.contains(brand)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Low Rated Product?
                        query = phone_data[phone_data['Company'].str.contains(brand)]
                        query = query.sort_values(by='Rating', ascending=True)
                        return query    
                        
            elif (len(num6)>0):
                print(len(contains_digit(query)))
                num,num2 = contains_digit(query)
                if (len(num)==1):
                    price = num[0]
                    if (contains_highest(query)):  # High Rated Product?
                        print("Highest")
                        query = phone_data[((phone_data['Company'].str.contains(brand))) & (phone_data['Price']>price)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Low Rated Product?
                        print("Lowest")
                        query = phone_data[((phone_data['Company'].str.contains(brand))) & (phone_data['Price']<price)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                elif (len(num)==2):
                    price1 = num[0]
                    price2 = num[1]
                    query = phone_data[((phone_data['Company'].str.contains(brand))) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                    query = query.sort_values(by='Price', ascending=False)
                    return query
            else:
                if(contains_lowest(query)):
                    query = phone_data[phone_data['Company'].str.contains(brand)]
                    query = query.sort_values(by=['Rating','Total Ratings'], ascending=True)
                    return query
                else:
                    query = phone_data[phone_data['Company'].str.contains(brand)]
                    query = query.sort_values(by='Rating', ascending=False)
                    return query
            
        elif(len(brand)==2):
            
            brand1 = brand[0]
            brand2 = brand[1] 
            if (contains_Rating(query)):          # Contains Rating?
                num,num2 = contains_digit(query)       # Checking if Rating specified or not
                if(len(num2)==1 and len(num)==0):                  # only rating given
                    rating = num2[0]
                    if (contains_highest(query)):    # Rating Higher?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']>rating)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Rating Lower?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']<rating)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    
                elif(len(num2)==1 and len(num)==1):     # Rating and Price Given
                    price = num[0]
                    rating = num2[0]
                    if (contains_highest(query)):   # Rating Higher?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']>rating) &  (phone_data['Price']<price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']>rating) &  (phone_data['Price']>price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                    elif (contains_lowest(query)):  # Rating Lower?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']<rating) &  (phone_data['Price']<price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']<rating) &  (phone_data['Price']>price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                
                elif(len(num)==2 and len(num2)==1):   # Rating + Price in between
                    
                    price1 = num[0]
                    price2 = num[1]
                    rating = num2[0]

                    if (contains_highest(query)):   # Rating Higher?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']>rating) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Price', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Rating Lower?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Rating']<rating) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Price', ascending=False)
                        return query
                        
                elif (len(num)==1 and len(num2)==0):
                    price = num[0]
                    if (contains_highest(query)):   # Rating Higher?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) &  (phone_data['Price']<price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) &  (phone_data['Price']>price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                    elif (contains_lowest(query)):  # Rating Lower?
                        cond = contains_price_low(query)
                        cond2 = contains_price_high(query)
                        if (cond==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) &  (phone_data['Price']<price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                        elif (cond2==True):
                            query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) &  (phone_data['Price']>price)]
                            query = query.sort_values(by='Rating', ascending=False)
                            return query
                            
                elif (len(num)==2 and len(num2)==0):
                    price1 = num[0]
                    price2 = num[1]

                    if (contains_highest(query)):   # Rating Higher?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Rating Lower?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    
                    
                else:    # Rating is not specified
                    
                    if (contains_highest(query)):  # High Rated Product?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2)))]
                        query = query.sort_values(by='Rating', ascending=False).head(5)
                        return query
                    elif (contains_lowest(query)):   # Low Rated Product?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2)))]
                        query = query.sort_values(by='Rating', ascending=True).head(5)
                        return query    
                        
            elif (len(num6)>0):
                num,num2 = contains_digit(query)
                if (len(num)==1):
                    price = num[0]
                    if (contains_highest(query)):  # High Rated Product?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Price']>price)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Low Rated Product?
                        query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & (phone_data['Price']<price)]
                        query = query.sort_values(by='Rating', ascending=True)
                        return query
                elif (len(num)==2):
                    price1 = num[0]
                    price2 = num[1]
                    query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2))) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                    query = query.sort_values(by='Price', ascending=False)
                    return query
            else:
                query = phone_data[((phone_data['Company'].str.contains(brand1)) | (phone_data['Company'].str.contains(brand2)))]
                query = query.sort_values(by='Rating', ascending=False)
                return query
    else:
        num1,num2 = contains_digit(query)
        if (contains_Rating(query)):
            if(len(num1)==0 and len(num2)==1): # only rating given
                rating = num2[0]
                if (contains_highest(query)):    # Rating Higher?
                    query = phone_data[(phone_data['Rating']>rating)]
                    query = query.sort_values(by='Rating', ascending=False)
                    return query
                elif (contains_lowest(query)):   # Rating Lower?
                    query = phone_data[(phone_data['Rating']<rating)]
                    query = query.sort_values(by='Rating', ascending=False)
                    return query
                    
            elif(len(num1)==1 and len(num2)==1):
                price = num1[0]
                rating = num2[0]
                if (contains_highest(query)):   # Rating Higher?
                    cond = contains_price_low(query)
                    cond2 = contains_price_high(query)
                    if (cond==True):
                        query = phone_data[(phone_data['Rating']>rating) &  (phone_data['Price']<price)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (cond2==True):
                        query = phone_data[(phone_data['Rating']>rating) &  (phone_data['Price']>price)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                elif (contains_lowest(query)):  # Rating Lower?
                    cond = contains_price_low(query)
                    cond2 = contains_price_high(query)
                    if (cond==True):
                        query = phone_data[(phone_data['Rating']<rating) &  (phone_data['Price']<price)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                    elif (cond2==True):
                        query = phone_data[(phone_data['Rating']<rating) &  (phone_data['Price']>price)]
                        query = query.sort_values(by='Rating', ascending=False)
                        return query
                        
            elif(len(num1)==2 and len(num2)==1):   # Rating + Price in between
                    
                    price1 = num1[0]
                    price2 = num1[1]
                    rating = num2[0]

                    if (contains_highest(query)):   # Rating Higher?
                        query = phone_data[(phone_data['Rating']>rating) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Price', ascending=False)
                        return query
                    elif (contains_lowest(query)):   # Rating Lower?
                        query = phone_data[(phone_data['Rating']<rating) & ((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                        query = query.sort_values(by='Price', ascending=False)
                        return query
                        
            elif(len(num1)==1 and len(num2)==0):
                price = num1[0]
                print(price)
                if (contains_price_high(query)):  # High Rated Product?
                    query = phone_data[(phone_data['Price']>price)]
                    query = query.sort_values(by=['Rating', 'Total Ratings'], ascending=[False, False])
                    return query
                elif (contains_price_low(query)):   # Low Rated Product?
                    query = phone_data[(phone_data['Price']<price)]
                    query = query.sort_values(by=['Rating', 'Total Ratings'], ascending=[False, False])
                    return query
                
            else:   
                    if (contains_highest(query)):  # High Rated Product?
                        query = phone_data.sort_values(by=['Rating', 'Total Ratings'], ascending=False).head(5)
                        return query
                    elif (contains_lowest(query)):   # Low Rated Product?
                        query = phone_data.sort_values(by=['Rating', 'Total Ratings'], ascending=True).head(5)
                        return query
                    
        elif (len(num1)>0):
            if (len(num1)==1):
                price = num1[0]
                if (contains_price_high(query)):  # High Rated Product?
                    query = phone_data[(phone_data['Price']>price)]
                    query = query.sort_values(by=['Rating', 'Total Ratings'], ascending=[False, False])
                    return query
                elif (contains_price_low(query)):   # Low Rated Product?
                    query = phone_data[(phone_data['Price']<price)]
                    query = query.sort_values(by=['Rating', 'Total Ratings'], ascending=[False, False])
                    return query
            elif (len(num1)==2):
                price1 = num1[0]
                price2 = num1[1]
                query = phone_data[((phone_data['Price']>price1) & (phone_data['Price']<price2))]
                query = query.sort_values(by=['Rating','Total Ratings'], ascending=False)
                return query               

In [12]:
def query_processor(query):
    query = clean_string(query)
    query = Display_Brands(query)
    return query

def query_processor2(query):
    query1 = clean_string2(query)
    query1 = phone_data[(phone_data['Product Name'].str.contains(query))]
    query1 = query1.sort_values(by='Rating', ascending=False)
    if(len(query1)==0):
        query = query_processor(query)
        return query
    else:
        
        return query1
    
    
    
        

In [14]:
x = 0
while(x<1):
    query = input('Please Enter your Query:')
    data = query_processor2(query)
    x = x + 1
data

Please Enter your Query: show me samsung and redmi phones with rating below 4.0


,Product ID,Company,Product Name,Price,Rating,Total Ratings,Total Questions Answered,Specifications,Link
63,64,redmi,"redmi note 12 - 6.67"" inch display - 8gb ram -...",50950,3.6,12,14,['model : redmi note 12 '],https://www.daraz.pk/products/redmi-note-12-66...
5,6,redmi,redmi note 12 8gb ram -128gb rom pta approved ...,50499,3.3,122,1,"['display6.67"" amoled dotdisplayrefresh rate: ...",https://www.daraz.pk/products/redmi-note-12-8g...
111,112,redmi,"redmi note 12s - 6.43"" inch display - 8gb ram ...",73450,1.0,1,0,['1 year brand warranty'],https://www.daraz.pk/products/redmi-note-12s-6...
194,195,samsung,"samsung galaxy a54 , 5g , 8gb ram 256gb rom , ...",135999,0.0,0,0,['buildosandroid 13 os uioneui 5.1 dimensions1...,https://www.daraz.pk/products/a54-5g-8gb-256gb...


In [ ]:
show me lower rated samsung phones

In [19]:

unique_product_ids = data['Product ID']
filtered_reviews = review_data[review_data['Product ID'].isin(unique_product_ids)]
review = int(input("Please Enter Product ID of the Phone of review "))
filtered_reviews = filtered_reviews[filtered_reviews['Product ID']==review]
filtered_reviews



Please Enter Product ID of the Phone of review  6


,Product ID,Review ID,Company,Review,Stars,Overall Review
75,6,1,Redmi,"phone is damaged from edges, pictures uploaded",1,Negative
76,6,2,Redmi,Same As Described original product found in re...,5,Positive
77,6,3,Redmi,Seller is fraud. He is sending Redmi 12C inste...,1,Negative
78,6,4,Redmi,Genuine product as shown. Most reasonable pric...,5,Neutral
79,6,5,Redmi,#11.11 Good and Pin Packed Mobile Thanks selle...,5,Positive
80,6,6,Redmi,#11.11 Ordered grey colour but received green....,5,Positive
81,6,7,Redmi,Same As Described original product found in re...,5,Positive
82,6,8,Redmi,"nice phone, camera results awesome 👍,",5,Positive
83,6,9,Redmi,thanks 👍 good product 👍 colour black kaha tha ...,5,Positive
84,6,10,Redmi,so beautiful so elegant just looking like a wa...,5,Positive
